In [0]:
dbutils.widgets.text("data_execucao", "")
data_execucao = dbutils.widgets.get("data_execucao")

In [0]:
from pyspark.sql.functions import lit
import requests

In [0]:
def extraindo_dados(date, base="BRL"):

  url = f"https://api.apilayer.com/exchangerates_data/{date}&base={base}"

  headers = {
  "apikey": "zWIlbtGaoigJGMj1Kdmwz3Ib4AHEVDAL"
  }

  # parametros = {"base":base,  "symbols": "USD,GBP,EUR, BRL"}
  parametros = {"base":base}
 
  response = requests.request("GET", url, headers=headers, params = parametros)

  if response.status_code != 200:
    raise Exception("Não consegui extrair dados")

  return response.json()

In [0]:
def create_df(dados):
  valores = [(moeda , float(taxa)) for moeda, taxa in dados['rates'].items()]
  return valores

In [0]:
def salvar_arquivo_parquet(conversao):
    ano, mes, dia  = conversao['date'].split('-')

    # path_moedas = f'dbfs:/databricks-results/bronze/{ano}/{mes}/{dia}'

    response = create_df(conversao) 

    df_moeadas = spark.createDataFrame(response, schema=['moeda', 'taxa'])
    df_moeadas = df_moeadas.withColumn('data', lit(f'{ano}-{mes}-{dia}'))

    # df_moeadas.write.format('parquet').mode('overwrite').save(path_moedas)

    # print(f"Arquivos salvos em {path_moedas}")
    return df_moeadas


## 2025-01-15

In [0]:
cotacoes = extraindo_dados(data_execucao)
# create_df(cotacoes)
s = salvar_arquivo_parquet(cotacoes)

In [0]:
s.show()

In [0]:

%sql

create table if not exists catalog_sql_teste.db_projeto.tb_taxa(
  moeda String, 
  taxa Float, 
  data date);

In [0]:
from pyspark.sql.functions import col

In [0]:
s = s.withColumn('taxa', col('taxa').cast('float'))
s = s.withColumn('data', col('data').cast('date'))

In [0]:
catalog = "catalog_sql_teste"
schema = "db_projeto"
table_name = "tb_taxa"
path = f'{catalog}.{schema}.{table_name}'

s.write.mode("overwrite").option('infoschema', 'true').saveAsTable(path)


In [0]:
%sql 
select * from catalog_sql_teste.db_projeto.tb_taxa